In [2]:
using Random
using Distributions
using FLoops
using Base.Threads
using BenchmarkTools

include("data.jl")
include("util.jl")

include("intervals/permutation.jl")
include("intervals/bootstrap.jl")
include("intervals/t.jl")

Threads.nthreads()

112

In [3]:
dtype = Float32
seed = 123

# DATA CONFIG

alpha = 0.05

# data
B  = 100   # num. coverage probabilities per boxplot
S  = 4300  # num. samples per coverage probability
nx = 8   # size of group 1
ny = 9    # size of group 2
MC = 10_000
perms = nothing
if binomial(nx + ny, nx) < 30_000
    perms = partition(nx, ny)
end

([1 1 … 9 10; 2 2 … 11 11; … ; 7 7 … 16 16; 8 9 … 17 17], [17 17 … 10 9; 16 16 … 8 8; … ; 10 10 … 2 2; 9 8 … 1 1])

In [4]:
# POPULATION SETTINGS

Random.seed!(123)

distrTypeX = Normal{dtype}
X_mu = random(Uniform(-1, 1), B)
X_sigma = random(Uniform(0.5, 1), B)
distrX = map(distrTypeX, X_mu, X_sigma)

distrTypeY = Normal{dtype}
Y_mu = X_mu
Y_sigma = random(Uniform(1.5, 2), B)
distrY = map(distrTypeY, Y_mu, Y_sigma)
;

In [5]:
deltas = @. mean(distrX) - mean(distrY)

@show distrX[1:2]
@show distrY[1:2]
@show deltas[1:2];

distrX[1:2] = Normal{Float32}[Normal{Float32}(μ=0.813f0, σ=0.977f0), Normal{Float32}(μ=-0.113f0, σ=0.923f0)]
distrY[1:2] = Normal{Float32}[Normal{Float32}(μ=0.813f0, σ=1.983f0), Normal{Float32}(μ=-0.113f0, σ=1.535f0)]
deltas[1:2] = Float32[0.0, 0.0]


In [6]:
Random.seed!(123)
xs = [dtype.(rand(distrX[i], nx, S)) for i in 1:B]
ys = [dtype.(rand(distrY[i], ny, S)) for i in 1:B]
@show size(ys)
ys[2]

size(ys) = (100,)


9×4300 Matrix{Float32}:
 -0.902477    -2.04466   -1.60487   …   2.11726    1.71138     0.0402462
 -1.44411     -0.112212   0.160702     -0.98992    0.0881085   2.29172
 -0.334489    -0.082353  -1.18844       1.82412    0.176726   -0.954524
 -0.00729625   0.945658  -1.78854      -0.42551    1.36516     0.811924
 -0.159706     1.25432   -1.67344      -3.14253    3.51393     1.18724
 -1.58751      0.662799   2.92263   …  -0.167795  -0.709806   -1.54364
  1.60311     -0.675307  -1.24478       1.62218    0.437874    1.10455
 -0.578301    -1.81976    0.460858     -3.55892    0.69047    -0.0107497
  0.260769     2.27429    0.133193      2.03919    1.94998     0.532423

In [7]:
# flatten into 3D matrix
X = reshape(hcat(xs...), nx, S, B)
Y = reshape(hcat(ys...), ny, S, B)
Y[:,:,2]

9×4300 Matrix{Float32}:
 -0.902477    -2.04466   -1.60487   …   2.11726    1.71138     0.0402462
 -1.44411     -0.112212   0.160702     -0.98992    0.0881085   2.29172
 -0.334489    -0.082353  -1.18844       1.82412    0.176726   -0.954524
 -0.00729625   0.945658  -1.78854      -0.42551    1.36516     0.811924
 -0.159706     1.25432   -1.67344      -3.14253    3.51393     1.18724
 -1.58751      0.662799   2.92263   …  -0.167795  -0.709806   -1.54364
  1.60311     -0.675307  -1.24478       1.62218    0.437874    1.10455
 -0.578301    -1.81976    0.460858     -3.55892    0.69047    -0.0107497
  0.260769     2.27429    0.133193      2.03919    1.94998     0.532423

In [8]:
function save_ci_results(results, methodId, B, S, pooled=nothing, two_sided=nothing; prefix="", dir="./")
    averages = []

    for batchId in 1:B
        batch = results[methodId, batchId, :]
        coverage = sum([j for (j, _) in batch]) / S
        width = sum([j for (_, j) in batch]) / S
        push!(averages, (coverage, width))
    end
    if isnothing(two_sided)
        save(averages, distrX[1:B], distrY[1:B], alpha, prefix=prefix, dir=dir)
    else
        alpha_ = two_sided ? alpha : alpha / 2
        save(averages, distrX[1:B], distrY[1:B], alpha_, pooled, two_sided, prefix=prefix, dir=dir)
    end
end

function save_permutation_results(results, B, S; prefix="", dir="./")
    i = 1
    per_method = []
    for two_sided in [true, false]
        for pooled in [true, false]
            save_ci_results(results, i, B, S, pooled, two_sided, prefix=prefix, dir=dir)
            i += 1
        end
    end
end

save_permutation_results (generic function with 1 method)

In [9]:
T = Threads.nthreads()
results = Array{Union{Tuple, Nothing}, 3}(nothing, 6, B, S)

#@time Threads.@threads for (i,j) in collect(Iterators.product(1:B, 1:S)) # 15.52 sec on (B,S) = (5, 1800)
@time @floop ThreadedEx(basesize=div(B*S, T)) for b in 1:B, s in 1:S
    @inbounds x = X[:,s,b]
    @inbounds y = Y[:,s,b]
 
    results[1, b, s] = permInterval(x, y, deltas[b], perms, true, alpha, twoSided, twoSided, 0.0005)    
    results[2, b, s] = permInterval(x, y, deltas[b], perms, false, alpha, twoSided, twoSided, 0.0005)
    results[3, b, s] = permInterval(x, y, deltas[b], perms, true, alpha/2, greater, smaller, 0.0005)
    results[4, b, s] = permInterval(x, y, deltas[b], perms, false, alpha/2, greater, smaller, 0.0005)
    
    """
    results[5, b, s] = bootstrap(x, y, deltas[b], alpha, nsamples=10_000)
    results[6, b, s] = tconf(x, y, deltas[b], alpha, false)
    """
end

dir = "../results/" * string(nx) * "_" * string(ny) * "/2/"
save_permutation_results(results, B, S; dir=dir)
#save_ci_results(results, 6, B, S; prefix="bs", dir=dir)

3429.800294 seconds (3.35 G allocations: 78.785 TiB, 51.59% gc time, 0.21% compilation time)
dir * filename = "../results/8_9/2/twoSided_pooled_0.05.csv"
dir * filename = "../results/8_9/2/twoSided_unpooled_0.05.csv"
dir * filename = "../results/8_9/2/oneSided_pooled_0.025.csv"
dir * filename = "../results/8_9/2/oneSided_unpooled_0.025.csv"
